In [56]:
import csv
import glob
import os
import os.path
from subprocess import call
from os import walk
import json
import random

In [57]:
rawframe = []
emoClassIndex = {'행복/즐거움': 0, '편안/안정': 1, '화남/불쾌': 2, '불안/슬픔': 3, '공격성': 4, '공포': 5}
emoClass = {'행복/즐거움': [], '편안/안정': [], '화남/불쾌': [], '불안/슬픔': [], '공격성': [], '공포': []}

emo_count1 = 0
emo_count2 = 0

for subdir, dirs, files in os.walk('./'):
    # json 읽어오기    
    if len(subdir) > 2 and subdir[2] == '2':                           
        for file_name in files:
            if file_name[-1] == 'n':    # 맨뒷글자 json의 n               
                json_path = subdir[2:] + "/" + file_name
                # json에서 감정 읽어오기
                dir = '/home/oem/data/5frames/' + json_path.split('-')[1] + '/' + json_path.split('/')[-1][:-5]
                #print(dir)
                frame_count = len(glob.glob(dir + "/*.jpg"))    # jpg 프레임 갯수 세기
                with open(json_path) as json_file:              # json 열고
                    data = json.load(json_file)                 # 읽고
                    emotion = data['inspect']['emotion']        # json 안에 inspect 안에 emotion ex.편안/안정
                    '''
                    # 천개 짜르기
                    #====================================================================================
                    if len(emoClass['행복/즐거움']) > 999 and emotion == '행복/즐거움':
                        continue
                    elif len(emoClass['편안/안정']) > 999 and emotion == '편안/안정':
                        continue
                    else:

                    #====================================================================================
                    '''
                    emoIndex = emoClassIndex[emotion]           # 감정 별로 번호 매기기
                    emoClass[emotion].append("{} {} {}".format(dir, frame_count, emoIndex))     # emoClass 안에 형식 맞춰서 저장    
                    

                    

                    
#print(emoClass)    
# 그러면 json에서 읽어온 감정(정답)별로 이미지 파일들이 나눠짐



In [58]:
split_rate = 0.8
trainRaw = {}
valRaw = {}          
arr = []
for clss in emoClass:           
    cls_len = len(emoClass[clss])
    split_point = int(split_rate * cls_len)  

    list = emoClass[clss]
    # list = [len(glob.glob(item[2:-4] + "/*.jpg")) for item in emoClass[clss]]
    # print(list)

    random.shuffle(list)                # 섞어주기
    trainRaw[clss] = list[:split_point] # 80%
    valRaw[clss] = list[split_point:]   # 20%
    # couting each emoClass
    arr.append([clss, len(emoClass[clss])])    
    print('{} : {}, train: {}, val {}'.format(clss, len(emoClass[clss]), len(trainRaw[clss]), len(valRaw[clss])))            


행복/즐거움 : 1000, train: 800, val 200
편안/안정 : 1000, train: 800, val 200
화남/불쾌 : 913, train: 730, val 183
불안/슬픔 : 780, train: 624, val 156
공격성 : 91, train: 72, val 19
공포 : 70, train: 56, val 14


In [59]:
with open('emoTrain01_pkn.txt', 'w') as f:
    for clss in trainRaw:
        for item in trainRaw[clss]:
            f.write("%s\n" % item)

In [60]:
with open('emoVal01_pkn.txt', 'w') as f:
    for clss in valRaw:
        for item in valRaw[clss]:
            f.write("%s\n" % item)